In [1]:
from queue import Queue
from threading import Thread, Lock, current_thread, Condition
import urllib.request
from bs4 import BeautifulSoup
import pymysql

In [2]:
class Producer(Thread):
    def __init__(self, lock, urls, queue):
        Thread.__init__(self)
        self._lock = lock
        self._urls = urls
        self._q = queue
    def run(self):
        while True:
            self._lock.acquire()
            print('in Producer {}'.format(current_thread()))
            if not(self._q.full()):
                if self._urls:
                    url = self._urls.pop()
                    data = urllib.request.urlopen(url).read().decode('UTF-8')
                    self._q.put(url+'!@#'+data)
                    self._lock.notify_all()
                else:
                    self._lock.release() 
                    break
            else:
                self._lock.wait()
            self._lock.release() 

In [3]:
class Consumer(Thread):
    def __init__(self, lock, queue, cursor, urls, db):
        Thread.__init__(self)
        self._lock = lock
        self._q = queue
        self._cursor = cursor
        self._urls = urls
        self._db = db
    def run(self):
        while True:
            self._lock.acquire()
            print('in Consumer {}'.format(current_thread()))
            if not(self._q.empty()):
                data = self._q.get()
                url = data.split('!@#')[0]
                data = data.split('!@#')[1]
                soup = BeautifulSoup(data, 'html.parser', from_encoding='utf8')
                title = soup.find_all(class_='main-title')[0].get_text()
                print(title)
                content = soup.find_all(class_='article')[0].get_text()
                sql_insert ="""insert into t1(url,title,content) values("{}","{}","{}")""".format(url,title,content)  
#                 print(sql_insert)
                try:  
                    cur.execute(sql_insert)  
                    #提交  
                    self._db.commit()  
                except Exception as e:  
                    #错误回滚  
                    print(e)
                    self._db.rollback() 
            else:
                if self._urls:
                    self._lock.notify_all()
                self._lock.wait()
                #cursor.execute('insert into t1(url,title,content) values(s%,s%,s%)',(self._url,title,data))
            self._lock.release() 

In [ ]:
myQueue = Queue(5)

In [ ]:
#l = Lock()
#打开数据库连接 
db= pymysql.connect(host="localhost",user="root",  
    password="123456",db="test",port=3306,charset="utf8") 
# 使用cursor()方法获取操作游标  
cur = db.cursor()  

l = Condition()
f = open('sina.txt')
urls = f.readlines()
producers = [Producer(l,urls,myQueue) for i in range(3)]
consumers = [Consumer(l,myQueue,cur,urls,db) for i in range(2)]
for p in producers:
    p.start()
for c in consumers:
    c.start()
for p in producers:
    p.join()
for c in consumers:
    c.join()
db.close()

in Producer <Producer(Thread-6, started 22880)>
in Producer <Producer(Thread-6, started 22880)>
in Producer <Producer(Thread-6, started 22880)>
in Producer <Producer(Thread-6, started 22880)>
in Producer <Producer(Thread-6, started 22880)>
in Producer <Producer(Thread-6, started 22880)>
in Producer <Producer(Thread-8, started 26860)>
in Consumer <Consumer(Thread-9, started 31820)>
默克尔：欧洲不能再指望美国保护 须自己掌握命运
in Consumer <Consumer(Thread-9, started 31820)>


C:\Users\Ian\Anaconda3\lib\site-packages\bs4\__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


外媒：马来西亚前总理纳吉布及其妻子被禁止离境
in Consumer <Consumer(Thread-9, started 31820)>
特朗普政府内部分歧不断 中美第二轮谈判或推迟
in Consumer <Consumer(Thread-9, started 31820)>
缅政府军与地方武装冲突致19死 中缅边境炮声明显
in Consumer <Consumer(Thread-9, started 31820)>
俄轰炸机时隔1年再逼近美国领土 美战机紧急拦截
in Consumer <Consumer(Thread-9, started 31820)>
in Producer <Producer(Thread-7, started 28676)>
in Producer <Producer(Thread-7, started 28676)>
in Producer <Producer(Thread-7, started 28676)>
in Producer <Producer(Thread-7, started 28676)>
in Producer <Producer(Thread-7, started 28676)>
in Producer <Producer(Thread-7, started 28676)>
in Producer <Producer(Thread-8, started 26860)>
in Consumer <Consumer(Thread-10, started 23412)>
朝坦克大赛规模大幅缩水 最高领导人金正恩未露面
in Consumer <Consumer(Thread-10, started 23412)>
外媒：伊朗以色列大规模交火 相互指责对方侵略
in Consumer <Consumer(Thread-10, started 23412)>
韩媒：朝鲜将于23-25日举行核试验场关闭仪式
in Consumer <Consumer(Thread-10, started 23412)>
哈萨克斯坦副总理访西安 中哈商定扩大农产品贸易
in Consumer <Consumer(Thread-10, started 23412)>
俄军方:俄轰炸机在中立水域飞行时遭美F22“伴飞"
(1064, "You